In [78]:
import csv
def partionnage(chemin):
    row=[]
    with open(chemin, 'r') as file:
        i=0
        reader = csv.reader(file, delimiter=';')
        new_row={}
        for row in reader:
            if(i!=0):
                temp_dic={}
                temp_dic["date"]=(row[0].split(","))[1].split(" ")[0]
                temp_dic["heure"]=int((row[0].split(","))[1].split(" ")[1].split(":")[0])
                temp_dic["OT"]=float((row[0].split(","))[2])
                new_row[int((row[0].split(","))[0])]=temp_dic
            i=i+1
        return new_row
    

data=partionnage('ETTh1_without_missing.csv')
data={cle: valeur for i, (cle, valeur) in enumerate(data.items()) if i > 7000}


In [79]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from datetime import datetime

#sans normalisation
def predictionNeuronne(data):
    
    dates = [datetime.strptime(value['date'], '%Y-%m-%d') for key, value in data.items()]
    years = [date.year for date in dates]
    months = [date.month for date in dates]
    days = [date.day for date in dates]
    hours = [value['heure'] for key, value in data.items()]
    ots = [value['OT'] for key, value in data.items()]
    
    features = np.column_stack((years, months, days, hours))  # Pas de normalisation
    test_start_index = len(features) - 100
    X_train = np.array(features[:test_start_index])
    y_train = np.array(ots[:test_start_index])
    X_test = np.array(features[test_start_index:])
    y_test = np.array(ots[test_start_index:])
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
    
    loss, mae = model.evaluate(X_test, y_test)
    print("mae=",model.evaluate(X_test, y_test)[1])
    headers = ["id", "Valeur attendu", "Valeur estime"] 
    
    predictions_scaled = model.predict(X_test)
    # Création des résultats pour chaque exemple
    resp = []
    for i in range(len(X_test)):
        id_val = 17219 + i
        actual_value = y_test[i]
        predicted_value = predictions_scaled[i]
        resp.append([id_val, actual_value, predicted_value])

    # Écriture des résultats dans un fichier CSV
    filename = "submissionReseauDeNeuronne.csv"
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["id", "Valeur attendue", "Valeur estimée"])
        writer.writerows(resp)

   
    

def pretraitementNewValue1(date,hour):
    years = int(date.split("-")[0])
    months = int(date.split("-")[1])
    days = int(date.split("-")[2])
    return  np.column_stack((years, months, days, hour))


def prediction(date,hour,data):
    
    features =pretraitementNewValue1(date,hour)
    predicted_scale=predictionNeuronne(data)
    return  predicted_scale.predict(features)
   

#sans normalisation

predictionNeuronne(data)


Epoch 1/10
256/256 [==============================] - 1s 2ms/step - loss: 297.2719 - mae: 10.0409 - val_loss: 11.2595 - val_mae: 2.6478
Epoch 2/10
256/256 [==============================] - 0s 1ms/step - loss: 51.5249 - mae: 6.1762 - val_loss: 17.2051 - val_mae: 3.4944
Epoch 3/10
256/256 [==============================] - 0s 1ms/step - loss: 51.2714 - mae: 6.1758 - val_loss: 8.8298 - val_mae: 2.2460
Epoch 4/10
256/256 [==============================] - 0s 1ms/step - loss: 52.8325 - mae: 6.2190 - val_loss: 7.4406 - val_mae: 2.1171
Epoch 5/10
256/256 [==============================] - 0s 1ms/step - loss: 52.3995 - mae: 6.2065 - val_loss: 7.5470 - val_mae: 2.1079
Epoch 6/10
256/256 [==============================] - 0s 1ms/step - loss: 60.2496 - mae: 6.4947 - val_loss: 36.1200 - val_mae: 5.4075
Epoch 7/10
256/256 [==============================] - 0s 1ms/step - loss: 61.8995 - mae: 6.5732 - val_loss: 8.5261 - val_mae: 2.3486
Epoch 8/10
256/256 [==============================] - 0s 1ms/ste

In [80]:
#en normalisant tout

def pretraitementNewValue(date, hour, scaler_year, scaler_month, scaler_day, scaler_hour):
    years = int(date.split("-")[0])
    months = int(date.split("-")[1])
    days = int(date.split("-")[2])
    hour_scaled = scaler_hour.transform([[hour]])
    return np.column_stack((scaler_year.transform([[years]]),
                            scaler_month.transform([[months]]),
                            scaler_day.transform([[days]]),
                            hour_scaled))
def predictionNeuronne(data):
    dates = [datetime.strptime(value['date'], '%Y-%m-%d') for key, value in data.items()]
    years = [date.year for date in dates]
    months = [date.month for date in dates]
    days = [date.day for date in dates]
    hours = [value['heure'] for key, value in data.items()]
    ots = [value['OT'] for key, value in data.items()]
    
    scaler_year = StandardScaler()
    scaler_month = StandardScaler()
    scaler_day = StandardScaler()
    scaler_hour = StandardScaler()
    scaler_ot = StandardScaler()
    
    years_scaled = scaler_year.fit_transform(np.array(years).reshape(-1, 1))
    months_scaled = scaler_month.fit_transform(np.array(months).reshape(-1, 1))
    days_scaled = scaler_day.fit_transform(np.array(days).reshape(-1, 1))
    hours_scaled = scaler_hour.fit_transform(np.array(hours).reshape(-1, 1))
    ots_scaled = scaler_ot.fit_transform(np.array(ots).reshape(-1, 1))
    
    features = np.column_stack((years_scaled, months_scaled, days_scaled, hours_scaled))
    
    features = np.column_stack((years, months, days, hours))  # Pas de normalisation
    test_start_index = len(features) - 100
    X_train = np.array(features[:test_start_index])
    y_train = np.array(ots[:test_start_index])
    X_test = np.array(features[test_start_index:])
    y_test = np.array(ots[test_start_index:])
    
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
    loss, mae = model.evaluate(X_test, y_test)
    print("mae=",model.evaluate(X_test, y_test)[1])
    headers = ["id", "Valeur attendu", "Valeur estime"] 
    # Evaluation du modèle sur l'ensemble de test
    predictions_scaled = model.predict(X_test)
    
    # Inverser la normalisation pour les prédictions de la variable OT

    # Création des résultats pour chaque exemple
    resp = []
    for i in range(len(X_test)):
        id_val = 17219 + i
        actual_value = y_test[i] 
        predicted_value = predictions_scaled[i]
        resp.append([id_val, actual_value, predicted_value])

    # Écriture des résultats dans un fichier CSV
    filename = "submissionReseauDeNeuronneNonNormalisee.csv"
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["id", "Valeur attendue", "Valeur estimée"])
        writer.writerows(resp)

predictionNeuronne(data)





Epoch 1/10
256/256 [==============================] - 1s 2ms/step - loss: 321.8839 - mae: 10.0575 - val_loss: 12.2986 - val_mae: 2.7271
Epoch 2/10
256/256 [==============================] - 0s 1ms/step - loss: 50.3323 - mae: 6.1291 - val_loss: 22.7042 - val_mae: 4.0554
Epoch 3/10
256/256 [==============================] - 0s 1ms/step - loss: 50.9638 - mae: 6.1456 - val_loss: 7.9700 - val_mae: 2.2248
Epoch 4/10
256/256 [==============================] - 0s 1ms/step - loss: 50.2992 - mae: 6.1349 - val_loss: 21.6654 - val_mae: 3.9543
Epoch 5/10
256/256 [==============================] - 0s 1ms/step - loss: 52.3452 - mae: 6.2277 - val_loss: 30.2651 - val_mae: 4.8602
Epoch 6/10
256/256 [==============================] - 0s 1ms/step - loss: 53.8857 - mae: 6.2967 - val_loss: 30.5751 - val_mae: 4.8924
Epoch 7/10
256/256 [==============================] - 0s 1ms/step - loss: 52.9408 - mae: 6.2184 - val_loss: 39.5753 - val_mae: 5.6982
Epoch 8/10
256/256 [==============================] - 0s 1ms/